# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/davidq/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/davidq/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  92995      0 --:--:-- --:--:-- --:--:-- 92839


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [23]:
from IPython.display import HTML, display
def set_output_wrapping():
    display(HTML('''
    <style>
    pre {
        white-space: pre-wrap;
        word-wrap: break-word;
        max-width: 100%;
        overflow-x: hidden;
    }
    .output_area {
        white-space: pre-wrap;
        word-wrap: break-word;
        max-width: 100%;
        overflow-x: hidden;
    }
    .output_text {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
    div.output {
        white-space: pre-wrap;
        word-wrap: break-word;
        max-width: 100%;
    }
    span {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
    </style>
    '''))
set_output_wrapping()

In [22]:
print(docs)

[Document(metadata={'source': 'data/2023_llms.html'}, page_content="Simon Willison’s Weblog\n\nSubscribe\n\nStuff we figured out about AI in 2023\n\n31st December 2023\n\n2023 was the breakthrough year for Large Language Models (LLMs). I think it’s OK to call these AI—they’re the latest and (currently) most interesting development in the academic field of Artificial Intelligence that dates back to the 1950s.\n\nHere’s my attempt to round up the highlights in one place!\n\nLarge Language Models\n\nThey’re actually quite easy to build\n\nYou can run LLMs on your own devices\n\nHobbyists can build their own fine-tuned models\n\nWe don’t yet know how to build GPT-4\n\nVibes Based Development\n\nLLMs are really smart, and also really, really dumb\n\nGullibility is the biggest unsolved problem\n\nCode may be the best application\n\nThe ethics of this space remain diabolically complex\n\nMy blog in 2023\n\nHere’s the sequel to this post: Things we learned about LLMs in 2024.\n\nLarge Language

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/davidq/projects/ai-makerspace/code/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/davidq/projects/ai-makerspace/code/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/davidq/projects/ai-makerspace/code/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 10, relationships: 27)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 10, relationships: 27)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=20, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/20 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What organization is included in the list of o...,[The ethics of this space remain diabolically ...,EleutherAI is mentioned among organizations su...,single_hop_specifc_query_synthesizer
1,Abu Dhabi what is it?,[The ethics of this space remain diabolically ...,The context mentions TII in Abu Dhabi as one o...,single_hop_specifc_query_synthesizer
2,What are the challenges associated with LLMs i...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The context mentions that LLMs are really smar...,single_hop_specifc_query_synthesizer
3,What are LLMs according to Simon Willison’s we...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"According to Simon Willison’s weblog, Large La...",single_hop_specifc_query_synthesizer
4,what is DALL-E 3,[the document includes some of the clearest ex...,the document does not provide a specific expla...,single_hop_specifc_query_synthesizer
5,What is Stanford Alpaca and how does it relate...,[the document includes some of the clearest ex...,Stanford Alpaca is mentioned in the context of...,single_hop_specifc_query_synthesizer
6,What is the significance of Claude in the cont...,[The year of slop Synthetic training data work...,"In the context provided, Claude refers to the ...",single_hop_specifc_query_synthesizer
7,What is DeepSeek in the context of AI models?,[The year of slop Synthetic training data work...,DeepSeek is listed among the organizations wit...,single_hop_specifc_query_synthesizer
8,What is Val Town?,[also pre-announced voice mode for Amazon Nova...,Val Town is mentioned as a platform where Stev...,single_hop_specifc_query_synthesizer
9,How does Cerebras relate to the development of...,[also pre-announced voice mode for Amazon Nova...,The provided context does not mention Cerebras...,single_hop_specifc_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=20)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/21 [00:00<?, ?it/s]

In [22]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wut has Meta dun in the feeld of LLMs this yeer?,[Code may be the best application The ethics o...,"In February, Meta released Llama, and in July,...",single_hop_specifc_query_synthesizer
1,What significant event related to AI ethics oc...,[Based Development As a computer scientist and...,"In September last year, the term 'prompt injec...",single_hop_specifc_query_synthesizer
2,Whaat are the key highlights from Simon Willis...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison's Weblog highlights that 2023 w...,single_hop_specifc_query_synthesizer
3,What role does Stanford Alpaca play in the dev...,[easy to follow. The rest of the document incl...,Stanford Alpaca is associated with the acceler...,single_hop_specifc_query_synthesizer
4,What are the ethical concerns associated with ...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns associated with the gulli...,multi_hop_abstract_query_synthesizer
5,Why are Large Language Models (LLMs) considere...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) are considered bl...,multi_hop_abstract_query_synthesizer
6,What are the ethical concerns related to the g...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the gullibilit...,multi_hop_abstract_query_synthesizer
7,How do the ethics of AI and the gullibility of...,[<1-hop>\n\nCode may be the best application T...,The ethics of AI and the gullibility of langua...,multi_hop_abstract_query_synthesizer
8,How has the development of models surpassing G...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the development of models surpassing ...",multi_hop_specific_query_synthesizer
9,How does the training efficiency of DeepSeek v...,[<1-hop>\n\nday after that. DeepSeek v3 is a h...,"DeepSeek v3, a 685B parameter model, is one of...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [19]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [24]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [31]:
list(client.list_examples(dataset_name=dataset_name, limit=3))[:3]

[<class 'langsmith.schemas.Example'>(id=bf005041-08da-4861-87ea-a61b1354bd33, dataset_id=2a53f468-8f56-4cfa-bb90-f3c5b1f42c97, link='https://smith.langchain.com/o/78cbe58d-b358-4331-abfb-293765485355/datasets/2a53f468-8f56-4cfa-bb90-f3c5b1f42c97/e/bf005041-08da-4861-87ea-a61b1354bd33'),
 <class 'langsmith.schemas.Example'>(id=3334488f-9fbe-4f16-a4b1-517c346ec253, dataset_id=2a53f468-8f56-4cfa-bb90-f3c5b1f42c97, link='https://smith.langchain.com/o/78cbe58d-b358-4331-abfb-293765485355/datasets/2a53f468-8f56-4cfa-bb90-f3c5b1f42c97/e/3334488f-9fbe-4f16-a4b1-517c346ec253'),
 <class 'langsmith.schemas.Example'>(id=1a782136-44f3-49d6-a938-e88938735998, dataset_id=2a53f468-8f56-4cfa-bb90-f3c5b1f42c97, link='https://smith.langchain.com/o/78cbe58d-b358-4331-abfb-293765485355/datasets/2a53f468-8f56-4cfa-bb90-f3c5b1f42c97/e/1a782136-44f3-49d6-a938-e88938735998')]

## Basic RAG Chain

Time for some RAG!


In [21]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [34]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/davidq/projects/ai-makerspace/code/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/davidq/projects/ai-makerspace/code/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [36]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [38]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [39]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague and frustrating term that lacks a single, clear, and widely understood meaning. Generally, it seems to converge on the idea of AI systems that can act on your behalf—akin to a travel agent model—or large language models (LLMs) given access to tools they can run in a loop to solve problems. However, despite much discussion and excitement, real, useful AI agents that operate autonomously in production are scarce or nonexistent so far. The term "agents" remains ambiguous, with many different definitions and no consensus on what it exactly entails.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [40]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [41]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [42]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'monthly-nail-37' at:
https://smith.langchain.com/o/78cbe58d-b358-4331-abfb-293765485355/datasets/2a53f468-8f56-4cfa-bb90-f3c5b1f42c97/compare?selectedSessions=b0527fb7-e976-404f-965f-312b5ed65397




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the themes of Llama 3 and Llama 3.3 rel...,I don't know.,None,The context highlights significant AI developm...,0,0,0,1.680406,bf005041-08da-4861-87ea-a61b1354bd33,43a85080-7af5-43b9-8cca-27bd0fc544f1
1,How does Meta's involvement in inference-scali...,"Based on the provided context, Meta is involve...",None,Meta published a paper on training large langu...,1,1,0,8.229812,3334488f-9fbe-4f16-a4b1-517c346ec253,0281fae9-2736-418b-a6a6-55ee836159ad
2,How do the explanations of how Large Language ...,The first segment explains that Large Language...,None,The first segment provides clear explanations ...,1,1,0,8.216067,1a782136-44f3-49d6-a938-e88938735998,ced7e805-294e-47e7-97dc-f52e6641659c
3,How do the recent developments in large langua...,"The context indicates that many organizations,...",None,The context highlights Google's significant ro...,1,1,0,5.082003,b7924581-d1d5-401d-b580-2e09a09c727c,21a924c2-2c84-4006-a7e6-3d37706b65fb
4,How does Google’s Gemini accept audio input an...,Google’s Gemini accepts audio input as part of...,None,Google’s Gemini accepts audio input and has a ...,1,1,0,2.933248,d7fb5ad8-72f4-4831-b242-62b55602a8d1,f3c3121a-d03c-43e6-9be4-55c3df8f432c
5,does Claude accept audio and is Claude like Ge...,"Based on the provided context, there is no dir...",None,"Yes, Claude is similar to Google Gemini in tha...",0,0,0,3.616056,27c0ea43-4ace-44df-970b-ea4008c0ed67,e499238b-e6b8-44ec-9896-20067fe21ace
6,Whos Simon Willison and what did he learn abou...,"Based on the provided context, Simon Willison ...",None,Simon Willison is a blogger who wrote about AI...,1,0,0,4.364111,9b0dc139-818f-4a57-b45f-85cd4182a725,5959d3bf-9be5-414d-af05-9e3da69570e9
7,How do misconceptions and misinformation about...,The context highlights that concerns about AI ...,None,The context highlights that while there is hyp...,1,1,0,3.499631,f495d520-29a2-43ea-9308-2ad539d23fda,19c0a62b-0424-4527-9b1f-23a0b3509a91
8,How do recent advancements in multi-modal AI m...,Recent advancements in multi-modal AI models l...,None,"Recent advancements in multi-modal AI models, ...",1,1,0,3.896515,dc6e77e3-b4ae-4cc8-a443-4d2c286f11bd,8a035ff4-caa9-4ce7-8512-b76883360478
9,AI in society and economy how impact on jobs a...,"Based on the provided context, AI has already ...",None,"The context discusses how AI models, including...",1,1,0,3.190793,04758cf0-6ac2-4aa3-a83a-d45a1b176aca,26601df6-f765-4737-9c11-751d5bfad920


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [43]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [44]:
rag_documents = docs

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [46]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [47]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [48]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [49]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [50]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Yo, agents are kinda like the holy grail in AI discussions but also the ultimate mystery — no single vibe defines them. Some folks think of agents like your classic travel agent, cruising around and handling business for you. Others see them as language models with tool access, looping through tasks to solve problems. It’s all tangled up with words like “autonomy,” but nobody drops a solid definition. \n\nBottom line? Agents are AI systems dreamed to act on your behalf, but real-deal examples are still ghostin’—more “coming soon” than “here now.” The biggest hitch? Gullibility. These agents gotta tell truth from fiction or else they’re just fancy fallible assistants.\n\nSo yeah, agents? That buzzword with mad potential but still waiting to truly flex. Keep your eyes peeled, ‘cause the future might just bring their glow-up!'

Finally, we can evaluate the new chain on the same test set!

In [51]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'impressionable-hair-55' at:
https://smith.langchain.com/o/78cbe58d-b358-4331-abfb-293765485355/datasets/2a53f468-8f56-4cfa-bb90-f3c5b1f42c97/compare?selectedSessions=a582c9ad-e669-4f51-bfbb-8beefab1af91




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the themes of Llama 3 and Llama 3.3 rel...,"Yo, gotta keep it real—based on the context yo...",None,The context highlights significant AI developm...,0,0,1,3.217474,bf005041-08da-4861-87ea-a61b1354bd33,0fa898ff-3c19-4e19-a235-b46b26591ae8
1,How does Meta's involvement in inference-scali...,"Alright, here’s the lowdown with some flair:\n...",None,Meta published a paper on training large langu...,1,0,1,5.305242,3334488f-9fbe-4f16-a4b1-517c346ec253,b278978c-1eb4-4b74-bdf4-00ce44ed3dcb
2,How do the explanations of how Large Language ...,"Alright, let’s break it down smooth and clear....",None,The first segment provides clear explanations ...,1,1,1,8.330624,1a782136-44f3-49d6-a938-e88938735998,cfadb1f4-015d-425a-8985-0a2c5bc525b5
3,How do the recent developments in large langua...,"Alright, here’s the lowdown, fresh and crispy:...",None,The context highlights Google's significant ro...,1,1,1,4.423395,b7924581-d1d5-401d-b580-2e09a09c727c,d0e6679b-517f-4122-a093-d9ad9c80928c
4,How does Google’s Gemini accept audio input an...,"Yo, here’s the lowdown straight from the conte...",None,Google’s Gemini accepts audio input and has a ...,1,0,1,6.039014,d7fb5ad8-72f4-4831-b242-62b55602a8d1,250f8a8e-7625-4bbf-937e-4be3b0490181
5,does Claude accept audio and is Claude like Ge...,"Alright, cool question! Based on the vibe of t...",None,"Yes, Claude is similar to Google Gemini in tha...",0,0,1,3.046396,27c0ea43-4ace-44df-970b-ea4008c0ed67,6cd9fdce-54d6-4124-b268-eeadba89c1dc
6,Whos Simon Willison and what did he learn abou...,"Alright, listen up! Simon Willison is the cool...",None,Simon Willison is a blogger who wrote about AI...,1,1,1,5.713223,9b0dc139-818f-4a57-b45f-85cd4182a725,e165f6b4-db04-41a9-8f3b-aabf23449209
7,How do misconceptions and misinformation about...,"Yo, here’s the lowdown straight from the conte...",None,The context highlights that while there is hyp...,1,1,1,4.400378,f495d520-29a2-43ea-9308-2ad539d23fda,504b051d-c933-4624-a35b-2672b5aa0396
8,How do recent advancements in multi-modal AI m...,"Yo, check this out — 2024 was straight-up the ...",None,"Recent advancements in multi-modal AI models, ...",1,1,1,6.320585,dc6e77e3-b4ae-4cc8-a443-4d2c286f11bd,07fe15f5-f420-4b3b-9354-e332a2617791
9,AI in society and economy how impact on jobs a...,"Aight, here’s the lowdown on AI’s scoop in job...",None,"The context discusses how AI models, including...",1,1,1,3.381242,04758cf0-6ac2-4aa3-a83a-d45a1b176aca,1431fd43-99dd-445e-b125-1cb13f225cbe


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.